In [1]:
import pandas as pd
import geopandas as gpd
import re
import matplotlib as plt
import numpy as np

In [2]:
trash = pd.read_csv('cleaned_trash_haulers.csv')

In [3]:
trash['Date Opened'] = pd.to_datetime(trash['Date Opened'])

In [4]:
trash['Zip Code'] = trash['Zip Code'].astype('str')
trash['Zip Code'] = trash['Zip Code'].str.extract(r'(\d{5})')

In [5]:
trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Request Number  20226 non-null  int64         
 1   Date Opened     20226 non-null  datetime64[ns]
 2   Request         20226 non-null  object        
 3   Description     20192 non-null  object        
 4   Zip Code        20186 non-null  object        
 5   Trash Hauler    19364 non-null  object        
 6   Trash Route     19279 non-null  object        
 7   LONGITUDE       20198 non-null  float64       
 8   LATITUDE        20198 non-null  float64       
 9   Clean Address   20216 non-null  object        
 10  Missed          20226 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 1.6+ MB


In [30]:
trash.Missed.value_counts()

Missed
True     17815
False     2411
Name: count, dtype: int64

# Red River Waste Solutions Total Damages
- For each address:
    - `>= 3 missed pickups on 3 unique dates`
    - `Within a 6 month timespan`
    + `+ $1500`

In [6]:
redriver = trash[['Request Number', 'Date Opened', 'Missed', 'Clean Address', 'Trash Hauler', 'Zip Code']]

In [7]:
redriver.head()

,Request Number,Date Opened,Missed,Clean Address,Trash Hauler,Zip Code
0,25270,2017-11-01,False,3817 CROUCH DR,RED RIVER,37207
1,25274,2017-11-01,True,4028 CLARKSVILLE PIKE,RED RIVER,37218
2,25276,2017-11-01,True,6528 THUNDERBIRD DR,RED RIVER,37209
3,25307,2017-11-01,True,2603 OLD MATTHEWS RD,WASTE IND,37207
4,25312,2017-11-01,True,604 CROLEY DR,RED RIVER,37209


In [8]:
redriver = redriver.loc[redriver['Trash Hauler'] == 'RED RIVER']

In [9]:
redriver = redriver.loc[redriver['Missed'] == True]

In [10]:
redriver.head()

,Request Number,Date Opened,Missed,Clean Address,Trash Hauler,Zip Code
1,25274,2017-11-01,True,4028 CLARKSVILLE PIKE,RED RIVER,37218
2,25276,2017-11-01,True,6528 THUNDERBIRD DR,RED RIVER,37209
4,25312,2017-11-01,True,604 CROLEY DR,RED RIVER,37209
8,25330,2017-11-01,True,4484 LAVERGNE COUCHVILLE PIKE,RED RIVER,37013
10,25341,2017-11-01,True,3113 HYDES FERRY RD,RED RIVER,37218


In [11]:
redriver = redriver.drop_duplicates(['Date Opened', 'Clean Address'])

In [12]:
#redriver['Time Diff'] = redriver.groupby('Clean Address')['Date Opened'].rolling(window = 3, min_periods = 3, center = True).DateOffset().reset_index(drop = True)

In [13]:
#redriver['timediff'] = redriver['Date Opened'].diff()

# Sort by date
# Make set of Addresses
- For loop for Address set
    - Make temporary df for address
    - For loop for dataframe
      - If firstdiff = date_opened - next row date opened
        - If first diff < 180 days
                - seconddiff = date_opened - 2nd next row date opened
                - If seconddiff < 180 days
                    - fine = True

In [14]:
redriver = redriver.sort_values('Date Opened').reset_index(drop = True)

In [15]:
address_set = set(redriver['Clean Address'])
address_set

{'3221 W HAMILTON AVE',
 '4409 DOWDY DR',
 '3257 ANDERSON RD',
 '1526 WAYNE DR',
 '1703 HORNBUCKLE LN',
 '3286 EZELL PIKE',
 '4149 HIGHLANDER CT',
 '504 EWING DR',
 '800 CAMMACK CT',
 '2013 CASTLEMAN DR',
 '3100 FORREST PARK AVE',
 '200 ERROL LN',
 '1004 ARCADIA CT',
 '4104 EDWARDS AVE',
 '114 ELIZABETHS CT',
 '2929 SOUTHLAKE DR',
 '1200 QUAISE MOOR E',
 '111 DODGE DR',
 '930 CURDWOOD BLVD',
 '1719 ROGER AVE',
 '206 MILES CT',
 '139 VOLCANO CT',
 '3603 SABRE DR',
 '473 RURAL HILL RD',
 '3908 W VALLEY DR',
 '3924 AUGUSTA DR',
 '4829 BRIARWOOD DR',
 '641 MAGNOLIA LN',
 '2024 CHADWELL OVERLOOK DR',
 '2316 WARFIELD LN',
 '5214 BUENA VISTA PIKE',
 '709 TUCKAHOE DR',
 '1515 BRIDGECREST DR',
 '2712 MEADOW ROSE DR',
 '522 BEN ALLEN RD',
 '5313 BALLARD CT',
 '213 GARNER AVE',
 '6531 CURRYWOOD DR',
 '200 LONGHUNTER CIR',
 '4604 LILA LN',
 '6005 OBRIEN AVE B',
 '787 RODNEY DR',
 '2715 PRIEST LAKE DR',
 '137 LAIRD RD',
 '4171 FARMVIEW DR',
 '1503 RIVERSIDE DR',
 '306 CANA CIR',
 '404 WESTCHESTER D

In [16]:
redriver['fine'] = False
#redriver['used_in_fine'] = False

In [17]:
for addy in address_set: # Iterates through each unique address
    addy_df = redriver.loc[redriver['Clean Address'] == addy].reset_index(drop = True) # Temp dataframe for each address
    for index, fields in addy_df.iterrows(): # Iterates through temp dataframe
        if len(addy_df) > 2: # Checks for at least 3 requests
          reqnum = fields['Request Number'] # Saves Request Number for reqeust
          if abs((addy_df.iloc[0, 1] - addy_df.iloc[1, 1]).days) <= 180: # Checks difference between rows 1 and 2
              if abs((addy_df.iloc[0, 1] - addy_df.iloc[2, 1]).days) <= 180: # Checks difference between rows 1 and 3
                  i = redriver[redriver['Request Number'] == reqnum].index # Saves index for Request Number in main dataframe
                  redriver.loc[i, 'fine'] = True
        addy_df = addy_df.drop(index=0).reset_index(drop = True)

In [18]:
redriver.head()

,Request Number,Date Opened,Missed,Clean Address,Trash Hauler,Zip Code,fine
0,25274,2017-11-01,True,4028 CLARKSVILLE PIKE,RED RIVER,37218,False
1,25529,2017-11-01,True,131 PAGE RD,RED RIVER,37205,False
2,25525,2017-11-01,True,162 ANTIOCH PIKE,RED RIVER,37211,True
3,25522,2017-11-01,True,405 BROOK HOLLOW RD,RED RIVER,37205,False
4,25519,2017-11-01,True,3702 HOBBS RD,RED RIVER,37215,True


In [26]:
redriver.fine.sum()

1435

In [28]:
redriver.loc[redriver['fine'] == True]['Zip Code'].nunique()

20

In [19]:
finedf = redriver

In [20]:
zipgroup = finedf.groupby('Zip Code')['fine'].sum().to_frame().reset_index()
zipgroup.head()

,Zip Code,fine
0,37013,220
1,37027,11
2,37086,0
3,37115,140
4,37189,10


In [29]:
zipgroup

,Zip Code,fine
0,37013,220
1,37027,11
2,37086,0
3,37115,140
4,37189,10
5,37204,38
6,37205,172
7,37206,9
8,37207,55
9,37208,2


In [21]:
zipcode = gpd.read_file('../geospatial-jaredjab/data/zipcodes.geojson')
zipcode.head()

,zip,objectid,po_name,shape_stlength,shape_starea,geometry
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-86.68725 36.31821, -86.68722 ..."
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-86.73451 36.23774, -86.73425 ..."
2,37204,9,NASHVILLE,93180.292250425613,200664795.51708984,"MULTIPOLYGON (((-86.77914 36.13424, -86.77923 ..."
3,37027,11,BRENTWOOD,159760.6942933173,174978422.04101562,"MULTIPOLYGON (((-86.81258 36.06319, -86.81263 ..."
4,37064,18,FRANKLIN,28995.828320601937,46969608.005737305,"MULTIPOLYGON (((-87.02197 36.01200, -87.02140 ..."


In [22]:
finezips = zipcode.merge(zipgroup, how='inner', left_on='zip', right_on='Zip Code')
finezips = finezips.drop(columns='Zip Code').rename(columns={'fine':'total_fines'})
finezips.head()

,zip,objectid,po_name,shape_stlength,shape_starea,geometry,total_fines
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-86.68725 36.31821, -86.68722 ...",140
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-86.73451 36.23774, -86.73425 ...",42
2,37204,9,NASHVILLE,93180.292250425613,200664795.51708984,"MULTIPOLYGON (((-86.77914 36.13424, -86.77923 ...",38
3,37027,11,BRENTWOOD,159760.6942933173,174978422.04101562,"MULTIPOLYGON (((-86.81258 36.06319, -86.81263 ...",11
4,37221,17,NASHVILLE,241372.3112694624,1266395059.4877319,"MULTIPOLYGON (((-86.97084 36.11644, -86.97084 ...",12


In [25]:
finezips.to_file('zipcode_fines_red_river.geojson', driver="GeoJSON") 

In [24]:
finetotal = 1500 * redriver['fine'].sum()
finetotal

2152500